In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("DataPush") \
    .config("spark.driver.extraClassPath", "/Users/eashvrudhula/PycharmProjects/Hackathon3/PostgresData/postgresql-42.6.0.jar").getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/12 17:19:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
hackathon_data = spark.read.csv("/Users/eashvrudhula/PycharmProjects/Hackathon3/PostgresData/hackathon_data.csv", header=True, inferSchema=True)

In [5]:
hackathon_data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- job_role: string (nullable = true)
 |-- job_description: string (nullable = true)
 |-- skills: string (nullable = true)
 |-- soft_skills: string (nullable = true)
 |-- responsibility: string (nullable = true)
 |-- location_name: string (nullable = true)
 |-- talent_cost: string (nullable = true)
 |-- currency_code: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- experience: string (nullable = true)
 |-- experience_range: string (nullable = true)
 |-- ready_for_application: string (nullable = true)


In [ ]:
def write_data_jdbc(df, database, table, environment, server=GATEWAY,
                        read_connection_config=read_postgres_config, **kwargs):
        env_dict = {"production": "prod",
                    "qa": "qa",
                    "development": "dev",
                    "dev": "dev",
                    "prod": "prod"
                    }
        environment = env_dict[environment]
        hostname, port, username, password, driver = read_connection_config(environment, server=server,
                                                                            database=database, access_mode="write")
        driver = "postgresql"

        url = "jdbc:{0}://{1}:{2}/{3}?user={4}&password={5}".format(driver, hostname, port, database, username,
                                                                    password)
        properties = {
            "user": username,
            "password": password,
            "driver": "org.postgresql.Driver"
        }
        df.write.jdbc(url=url, table=table, mode='append', properties=properties)

In [6]:
hackathon_data.select("skills").show(10,False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|skills                                                                                                                                                                                                                                                                                                                                                                               |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [10]:
import pyspark.sql.functions as f
hackathon_data.withColumn("skills", f.split("skills",";")).withColumn("soft_skills", f.split("soft_skills",";")).select("skills","soft_skills").show(50,False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------+
|skills                                                                                                                                                                                                                                                                                                                                                                                                         |soft_skills                               

In [11]:
hackathon_data_modified = hackathon_data.withColumn("skills", f.split("skills",";")).withColumn("soft_skills", f.split("soft_skills",";"))

In [12]:
hackathon_data_modified.write.parquet("/Users/eashvrudhula/PycharmProjects/Hackathon3/PostgresData/hackathon_data_modified")

23/10/12 16:46:30 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/10/12 16:46:30 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/10/12 16:46:30 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , job_role, job_description, skills, soft_skills, responsibility, location_name, talent_cost, currency_code, currency, experience, experience_range, ready_for_application
 Schema: _c0, job_role, job_description, skills, soft_skills, responsibility, location_name, talent_cost, currency_code, currency, experience, experience_range, ready_for_application
Expected: _c0 but found: 
CSV file: file:///Users/eashvrudhula/PycharmProjects/Hackathon3/PostgresData/hackathon_data.csv
23/10/12 16:47:00 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group si

In [13]:
print("hi")

hi


In [3]:
modded = spark.read.parquet("/Users/eashvrudhula/PycharmProjects/Hackathon3/PostgresData/hackathon_data_modified")

In [23]:
modded.write.format("csv").save("/Users/eashvrudhula/PycharmProjects/Hackathon3/PostgresData/hackathon_data_modified_csv",header=True)

AnalysisException: [UNSUPPORTED_DATA_TYPE_FOR_DATASOURCE] The CSV datasource doesn't support the column `skills` of the type "ARRAY<STRING>".

In [16]:
modded.count()

28612886

In [ ]:
28612886

In [17]:
hackathon_data.count()

28612886

In [5]:
url = "localhost:5432/postgres?user=postgres&password=$Eash1312#"

In [18]:
# properties = {
#             "user": "postgres",
#             "password": "$Eash1312#",
#             "driver": "org.postgresql.Driver"
# }

NameError: name 'url' is not defined

In [22]:
properties = {
            "user": "postgres",
            "password": "$Eash1312#",
            "driver": "org.postgresql.Driver"
}
driver = "postgresql"
hostname = "localhost"
port = "5432"
database = "postgres"
username = "postgres"
password = "$Eash1312#"
url = "jdbc:{0}://{1}:{2}/{3}?user={4}&password={5}".format(driver, hostname, port, database, username,
                                                                    password)

In [23]:
skillset.drop("_c0").write.jdbc(url=url, table="job_data", mode='append',properties=properties)

23/10/12 17:49:11 ERROR Executor: Exception in task 2.0 in stage 12.0 (TID 40)8]
java.sql.BatchUpdateException: Batch entry 0 INSERT INTO job_data ("job_role","job_description","responsibility","location_name","talent_cost","currency_code","currency","experience","experience_range","ready_for_application","skillset") VALUES (NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,'{"soft_skills": null, "skills": null}') was aborted: ERROR: column "talent_cost" is of type numeric but expression is of type character varying
  Hint: You will need to rewrite or cast the expression.
  Position: 214  Call getNextException to see other errors in the batch.
	at org.postgresql.jdbc.BatchResultHandler.handleError(BatchResultHandler.java:165)
	at org.postgresql.jdbc.PgStatement.internalExecuteBatch(PgStatement.java:880)
	at org.postgresql.jdbc.PgStatement.executeBatch(PgStatement.java:919)
	at org.postgresql.jdbc.PgPreparedStatement.executeBatch(PgPreparedStatement.java:1685)
	at org.apache.spark.sql.e

Py4JJavaError: An error occurred while calling o243.jdbc.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 12.0 failed 1 times, most recent failure: Lost task 4.0 in stage 12.0 (TID 42) (192.168.2.137 executor driver): java.sql.BatchUpdateException: Batch entry 0 INSERT INTO job_data ("job_role","job_description","responsibility","location_name","talent_cost","currency_code","currency","experience","experience_range","ready_for_application","skillset") VALUES (NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,'{"soft_skills": null, "skills": null}') was aborted: ERROR: column "talent_cost" is of type numeric but expression is of type character varying
  Hint: You will need to rewrite or cast the expression.
  Position: 214  Call getNextException to see other errors in the batch.
	at org.postgresql.jdbc.BatchResultHandler.handleError(BatchResultHandler.java:165)
	at org.postgresql.jdbc.PgStatement.internalExecuteBatch(PgStatement.java:880)
	at org.pos

In [17]:
print(driver)

NameError: name 'driver' is not defined

In [10]:
modded_df = modded.toPandas()

23/10/12 17:17:11 ERROR Executor: Exception in task 2.0 in stage 1.0 (TID 3)/ 8]
java.lang.OutOfMemoryError: Java heap space
	at java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:57)
	at java.nio.ByteBuffer.allocate(ByteBuffer.java:335)
	at org.apache.spark.serializer.SerializerHelper$.$anonfun$serializeToChunkedBuffer$1(SerializerHelper.scala:40)
	at org.apache.spark.serializer.SerializerHelper$.$anonfun$serializeToChunkedBuffer$1$adapted(SerializerHelper.scala:40)
	at org.apache.spark.serializer.SerializerHelper$$$Lambda$1644/39752490.apply(Unknown Source)
	at org.apache.spark.util.io.ChunkedByteBufferOutputStream.allocateNewChunkIfNeeded(ChunkedByteBufferOutputStream.scala:87)
	at org.apache.spark.util.io.ChunkedByteBufferOutputStream.write(ChunkedByteBufferOutputStream.scala:75)
	at java.io.ObjectOutputStream$BlockDataOutputStream.write(ObjectOutputStream.java:1853)
	at java.io.ObjectOutputStream.write(ObjectOutputStream.java:709)
	at org.apache.spark.util.Utils$.$anonfun$writeByt

Py4JJavaError: An error occurred while calling o30.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 1.0 failed 1 times, most recent failure: Lost task 2.0 in stage 1.0 (TID 3) (192.168.2.137 executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:57)
	at java.nio.ByteBuffer.allocate(ByteBuffer.java:335)
	at org.apache.spark.serializer.SerializerHelper$.$anonfun$serializeToChunkedBuffer$1(SerializerHelper.scala:40)
	at org.apache.spark.serializer.SerializerHelper$.$anonfun$serializeToChunkedBuffer$1$adapted(SerializerHelper.scala:40)
	at org.apache.spark.serializer.SerializerHelper$$$Lambda$1644/39752490.apply(Unknown Source)
	at org.apache.spark.util.io.ChunkedByteBufferOutputStream.allocateNewChunkIfNeeded(ChunkedByteBufferOutputStream.scala:87)
	at org.apache.spark.util.io.ChunkedByteBufferOutputStream.write(ChunkedByteBufferOutputStream.scala:75)
	at java.io.ObjectOutput

In [4]:
modded.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- job_role: string (nullable = true)
 |-- job_description: string (nullable = true)
 |-- skills: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- soft_skills: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- responsibility: string (nullable = true)
 |-- location_name: string (nullable = true)
 |-- talent_cost: string (nullable = true)
 |-- currency_code: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- experience: string (nullable = true)
 |-- experience_range: string (nullable = true)
 |-- ready_for_application: string (nullable = true)


In [8]:
import json
import pyspark.sql.functions as f
def makejsonstring(soft_skills,skills):
    return json.dumps({"soft_skills":soft_skills,"skills":skills})
udf_makejsonstring = f.udf(makejsonstring)
modded.filter(f.col("soft_skills").isNotNull()).withColumn("skillset", udf_makejsonstring("soft_skills","skills")).select("skillset").show(100,False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|skillset                                                                                                                                                                                                                                                                                                            

In [10]:
skillset = modded.withColumn("skillset", udf_makejsonstring("soft_skills","skills"))

In [11]:
skillset.withColumn("skillset_length", f.length("skillset")).select("skillset_length").orderBy(f.col("skillset").desc()).show(100,False)

+---------------+
|skillset_length|
+---------------+
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37             |
|37       

In [12]:
skillset = modded.withColumn("skillset", udf_makejsonstring("soft_skills","skills")).drop("skills","soft_skills")

In [15]:
skillset.drop("_c0").repartition(1).write.format("csv").mode("overwrite").save("/Users/eashvrudhula/PycharmProjects/Hackathon3/PostgresData/hackathon_data_modified_csv_new",header=True)

In [16]:
skillset.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- job_role: string (nullable = true)
 |-- job_description: string (nullable = true)
 |-- responsibility: string (nullable = true)
 |-- location_name: string (nullable = true)
 |-- talent_cost: string (nullable = true)
 |-- currency_code: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- experience: string (nullable = true)
 |-- experience_range: string (nullable = true)
 |-- ready_for_application: string (nullable = true)
 |-- skillset: string (nullable = true)


In [24]:
skillset = spark.read.format("csv").load("/Users/eashvrudhula/PycharmProjects/Hackathon3/PostgresData/hackathon_data_modified_csv_new",header=True)

In [26]:
skillset.printSchema()

root
 |-- job_role: string (nullable = true)
 |-- job_description: string (nullable = true)
 |-- responsibility: string (nullable = true)
 |-- location_name: string (nullable = true)
 |-- talent_cost: string (nullable = true)
 |-- currency_code: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- experience: string (nullable = true)
 |-- experience_range: string (nullable = true)
 |-- ready_for_application: string (nullable = true)
 |-- skillset: string (nullable = true)


In [27]:
properties = {
            "user": "postgres",
            "password": "$Eash1312#",
            "driver": "org.postgresql.Driver"
}
driver = "postgresql"
hostname = "localhost"
port = "5432"
database = "postgres"
username = "postgres"
password = "$Eash1312#"
url = "jdbc:{0}://{1}:{2}/{3}?user={4}&password={5}".format(driver, hostname, port, database, username,
                                                                    password)

In [30]:
skillset.select("talent_cost").filter(f.col("talent_cost").isNotNull()).show(100,False)

+----------------------------+
|talent_cost                 |
+----------------------------+
|False                       |
|True                        |
|False                       |
|False                       |
|False                       |
|False                       |
|True                        |
|False                       |
|False                       |
|False                       |
|True                        |
|True                        |
|True                        |
|False                       |
|False                       |
|False                       |
|False                       |
|True                        |
|False                       |
|True                        |
|False                       |
|False                       |
|False                       |
|False                       |
|True                        |
|False                       |
|False                       |
|True                        |
|False                       |
|False  

In [31]:
skillset.printSchema()

root
 |-- job_role: string (nullable = true)
 |-- job_description: string (nullable = true)
 |-- responsibility: string (nullable = true)
 |-- location_name: string (nullable = true)
 |-- talent_cost: string (nullable = true)
 |-- currency_code: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- experience: string (nullable = true)
 |-- experience_range: string (nullable = true)
 |-- ready_for_application: string (nullable = true)
 |-- skillset: string (nullable = true)


In [32]:
hackathon_data_og = spark.read.format("csv").load("/Users/eashvrudhula/PycharmProjects/Hackathon3/PostgresData/hackathon_data.csv",header=True)

In [34]:
hackathon_data_og.select("talent_cost").filter(f.col("talent_cost").isNotNull()).show(100,False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------+
|talent_cost                                                                                                                                              |
+---------------------------------------------------------------------------------------------------------------------------------------------------------+
|True                                                                                                                                                     |
|True                                                                                                                                                     |
|True                                                                                                                                                     |
|True                                                           

In [35]:
hackdata = spark.read.parquet("/Users/eashvrudhula/PycharmProjects/Hackathon3/HackData/part-00000-7ffe30f6-3f37-461b-9f30-5b3f9f1e5600-c000.snappy.parquet")

In [36]:
hackdata.printSchema()

root
 |-- job_role: string (nullable = true)
 |-- job_description: string (nullable = true)
 |-- skills: string (nullable = true)
 |-- soft_skills: string (nullable = true)
 |-- responsibility: string (nullable = true)
 |-- location_name: string (nullable = true)
 |-- talent_cost: double (nullable = true)
 |-- currency_code: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- experience: double (nullable = true)
 |-- experience_range: string (nullable = true)


In [37]:
hackdata.select("talent_cost").filter(f.col("talent_cost").isNotNull()).show(100,False)

+--------------------+
|talent_cost         |
+--------------------+
|219880.34011299434  |
|127928.70406779661  |
|1.2655719215368384E7|
|61797.6             |
|72000.0             |
|120000.0            |
|95000.0             |
|57000.0             |
|88200.0             |
|1.177922353526969E7 |
|1.2302447536025774E7|
|1.2898235449295474E7|
|120000.0            |
|72000.0             |
|56160.0             |
|9057122.027686495   |
|216000.0            |
|129600.0            |
|43200.0             |
|72000.0             |
|145497.6            |
|242496.0            |
|108000.0            |
|64800.0             |
|86400.0             |
|144000.0            |
|121998.0            |
|73198.8             |
|78300.0             |
|130500.0            |
|60000.0             |
|36000.0             |
|1.1102278614583446E7|
|1.1639824440177742E7|
|1.0629658394922385E7|
|216000.0            |
|129600.0            |
|100000.0            |
|60000.0             |
|50400.0             |
|84000.0   

In [43]:
import json
import pyspark.sql.functions as f
def makejsonstring(soft_skills,skills):
    return json.dumps({"soft_skills":soft_skills,"skills":skills})
udf_makejsonstring = f.udf(makejsonstring)
hackdata.withColumn("skills", f.split("skills",";")).withColumn("soft_skills", f.split("soft_skills",";")).withColumn("skillset", udf_makejsonstring("soft_skills","skills")).drop("skills","soft_skills").repartition(1).write.mode("overwrite").format("csv").save("/Users/eashvrudhula/PycharmProjects/Hackathon3/PostgresData/hackathon_data_modified_csv_new",header=True)

In [50]:
df.write.jdbc(url=url, table="job_data", mode='append',properties=properties)

In [45]:
hackdata.withColumn("skills", f.split("skills",";")).withColumn("soft_skills", f.split("soft_skills",";")).withColumn("skillset", udf_makejsonstring("soft_skills","skills")).drop("skills","soft_skills").write.mode("overwrite").parquet("/Users/eashvrudhula/PycharmProjects/Hackathon3/PostgresData/hackathon_data_modified")

In [46]:
df = spark.read.parquet("/Users/eashvrudhula/PycharmProjects/Hackathon3/PostgresData/hackathon_data_modified")

In [49]:
df.select("experience").show(100,False)

+------------------+
|experience        |
+------------------+
|NULL              |
|NULL              |
|4.96              |
|5.865000076293946 |
|4.200000076293946 |
|NULL              |
|4.874999961853027 |
|4.720000228881836 |
|6.399999923706055 |
|3.5750000762939456|
|5.755000076293945 |
|NULL              |
|6.07              |
|4.28              |
|4.630000076293945 |
|5.925000152587891 |
|4.200000076293946 |
|4.96              |
|4.96              |
|4.380000076293945 |
|4.585             |
|4.449999771118164 |
|5.539999923706055 |
|4.289999923706055 |
|4.970000228881836 |
|4.96              |
|5.470000228881836 |
|4.170000038146973 |
|4.28              |
|NULL              |
|NULL              |
|NULL              |
|3.0               |
|NULL              |
|NULL              |
|NULL              |
|NULL              |
|3.0               |
|NULL              |
|NULL              |
|3.0               |
|NULL              |
|NULL              |
|NULL              |
|NULL        

In [97]:
company_data_df = spark.read.format("csv").load("/Users/eashvrudhula/PycharmProjects/Hackathon3/PostgresData/company_synon_descriptions.csv",header = True)

In [98]:
company_data_df.printSchema()

root
 |-- company_name: string (nullable = true)
 |-- description: string (nullable = true)


In [114]:
properties = {
            "user": "postgres",
            "password": "$Eash1312#",
            "driver": "org.postgresql.Driver"
}
driver = "postgresql"
hostname = "localhost"
port = "5432"
database = "postgres"
username = "postgres"
password = "JDCrafters"
url = "jdbc:{0}://{1}:{2}/{3}?user={4}&password={5}".format(driver, hostname, port, database, username, password)

In [99]:
company_data_df.write.jdbc(url=url, table="company_description_table", mode='append',properties=properties)


In [108]:
table = "company_description_table"
# query = "SELECT * FROM {0}".format(table)
query = "SELECT company_name,description from company_description_table WHERE company_name ILIKE '%google%'"
company_data_df_read = spark.read.format('jdbc').options(
                url=url,
                database=database,
                dbtable="({0}) as t".format(query) if query else "{0}.{1}".format(database, table),
            ).load()

In [109]:
company_data_df_read.show(100,False)

+------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|company_name|description                                                                                                                                                                                                                                                                                                                                                                                           |
+------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [87]:
glassdoor = spark.read.format("csv").load("/Users/eashvrudhula/PycharmProjects/Hackathon3/PostgresData/hackathon_glassdoor_data.csv",header = True)

In [88]:
glassdoor.drop("").printSchema()

root
 |-- _c0: string (nullable = true)
 |-- job_title_lemmatized1: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- job_title_lemmatized3: string (nullable = true)
 |-- jobTitle: string (nullable = true)
 |-- cultureAndValuesRating: string (nullable = true)
 |-- compensationAndBenefitsRating: string (nullable = true)
 |-- ceoApproval: string (nullable = true)
 |-- isFeaturedReview: string (nullable = true)
 |-- workLifeBalanceRating: string (nullable = true)
 |-- overallNumeric: string (nullable = true)
 |-- overall: string (nullable = true)
 |-- synonimised_company_name: string (nullable = true)
 |-- job_role: string (nullable = true)


In [92]:
glassdoor.drop("_c0","job_title_lemmatized3").withColumnRenamed("job_title_lemmatized1","job_title_lemmatized").write.jdbc(url=url, table="glassdoor_reviews", mode='append',properties=properties)

23/10/12 19:11:11 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: job_title_lemmatized, company_name, jobTitle, cultureAndValuesRating, compensationAndBenefitsRating, ceoApproval, isFeaturedReview, workLifeBalanceRating, overallNumeric, overall, synonimised_company_name, job_role
 Schema: job_title_lemmatized1, company_name, jobTitle, cultureAndValuesRating, compensationAndBenefitsRating, ceoApproval, isFeaturedReview, workLifeBalanceRating, overallNumeric, overall, synonimised_company_name, job_role
Expected: job_title_lemmatized1 but found: job_title_lemmatized
CSV file: file:///Users/eashvrudhula/PycharmProjects/Hackathon3/PostgresData/hackathon_glassdoor_data.csv


In [118]:
table = "job_data"
query = "SELECT * FROM {0}".format(table)
company_data_df_read = spark.read.format('jdbc').options(
                url=url,
                database=database,
                dbtable="({0}) as t".format(query) if query else "{0}.{1}".format(database, table),
            ).load()

In [119]:
company_data_df_read.printSchema()


root
 |-- id: integer (nullable = true)
 |-- job_role: string (nullable = true)
 |-- job_description: string (nullable = true)
 |-- skillset: string (nullable = true)
 |-- responsibility: string (nullable = true)
 |-- location_name: string (nullable = true)
 |-- talent_cost: decimal(38,18) (nullable = true)
 |-- currency_code: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- experience: decimal(38,18) (nullable = true)
 |-- experience_range: string (nullable = true)
 |-- ready_for_application: boolean (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)


In [111]:
a = company_data_df_read.rdd.map(lambda x: x.asDict()).collect()[0]

In [112]:
print(a)

{'company_name': 'Google Inc.', 'description': "Google Inc., a subsidiary of Alphabet Inc., is a multinational technology company known for its internet-related services and products. Google offers a wide range of services, including search engine, online advertising technologies, cloud computing, software applications, and hardware devices. They strive to organize the world's information and make it universally accessible and useful."}


In [121]:
queries = {
    "responsibilites_and_skills": "a",
    "about_company": "b",
    "salary":"c"
}

In [124]:
for key_names in queries:
    print(key_names)
    print(queries[key_names])

responsibilites_and_skills
a
about_company
b
salary
c


In [125]:
response_json = {
    "about_company": [
        [
            226193,
            "Google Inc.",
            "Google Inc., a subsidiary of Alphabet Inc., is a multinational technology company known for its internet-related services and products. Google offers a wide range of services, including search engine, online advertising technologies, cloud computing, software applications, and hardware devices. They strive to organize the world's information and make it universally accessible and useful."
        ]
    ],
    "responsibilites_and_skills": [
        [
            "DevOps Engineer",
            "The role is responsible for designing, developing, and maintaining CI/CD pipelines to automate software build, testing, deployment, and monitoring processes. The role collaborates with development teams to understand application requirements, architecture, and infrastructure needs to fullfill the needs without any correction. The role monitors and maintain production systems, troubleshoot issues, and perform root cause analysis to prevent future incidents.",
            "Deploy, automate, maintaining, and manage cloud-based system, to ensure the availability, performance, scalability, and security of production systems; Investigate and resolve technical issues by deploying updates and fixes; Collect and review customers feedback to enhance user experience; Update the processes and design new processes as needed to optimize performance; Test system integrity, implement designs, application developments, and other processes related to infrastructure, making improvements as needed; Manage and support integration of new technologies; Ensure that systems are safe and secure against cybersecurity threats;\n<p><strong>Emerging Functional Skills</strong></p>\n<ul>\n<li>Knowledge of Scripting Languages such as <strong>Bash</strong>, <strong>Powershell</strong></li>\n<li>Experience in <strong>Agile Methodologies</strong></li>\n<li>Familiarity with Database Management Software tool like <strong>SQL</strong></li>\n<li>Experience in Programming Languages such a <strong>Java</strong>, <strong>Python</strong></li>\n<li>Knowledge of Cloud technologies tools such as <strong>Amazon Web Services</strong> (AWS)</li>\n</ul>;\n<p><strong>Key Behavioural Skills</strong></p>\n<ul>\n<li><strong>Analytical</strong></li>\n<li>collaborative</li>\n<li><strong>Critical Thinking</strong></li>\n<li>Integrity</li>\n<li>Multitasking</li>\n</ul>",
            "{\"soft_skills\": [\"Collaboration\", \" Flexibility\", \" Multitasking\", \" Planning\", \" Client Handling\", \" Supportive\", \" Problem Solving\", \" Business Acumen\", \" Quality Orientation\", \" Tenacity\"], \"skills\": [\"Software Development\", \" Scripting\", \" Git\", \" Apache Subversion\", \" Jenkins\", \" Travis CI\", \" Splunk\", \" Server Operations\", \" Ansible\", \" Saltstack\", \" VMware ESXi\", \" OpenStack\", \" Xen\"]}"
        ]
    ]
}

In [127]:
response_json["responsibilites_and_skills"][0][1]

'The role is responsible for designing, developing, and maintaining CI/CD pipelines to automate software build, testing, deployment, and monitoring processes. The role collaborates with development teams to understand application requirements, architecture, and infrastructure needs to fullfill the needs without any correction. The role monitors and maintain production systems, troubleshoot issues, and perform root cause analysis to prevent future incidents.'

In [128]:
responsibilites_and_skills = response_json.get('responsibilites_and_skills', None)

In [132]:
responsibilites_and_skills[0][0]

'DevOps Engineer'

In [133]:
if responsibilites_and_skills:
    job_title = responsibilites_and_skills[0][0]
    job_role_description = responsibilites_and_skills[0][1]
    responsibility = responsibilites_and_skills[0][2]
    skillset = responsibilites_and_skills[0][3]
    core_skills = json.loads(skillset)["skills"]
    soft_skills = json.loads(skillset)["soft_skills"]

In [134]:
job_role_description

'The role is responsible for designing, developing, and maintaining CI/CD pipelines to automate software build, testing, deployment, and monitoring processes. The role collaborates with development teams to understand application requirements, architecture, and infrastructure needs to fullfill the needs without any correction. The role monitors and maintain production systems, troubleshoot issues, and perform root cause analysis to prevent future incidents.'

In [ ]:
responsibility